# HW1

In [ ]:
import pandas as pd

In [3]:
data = {0:[0,0,0,0,0,0,0,0], 1: [0,2,0,1,0,0,0,0], 2:[0,0,2,0,0,1,1,0], 3: [0,2,0,1,1,3,0,0], 4: [0,1,2,3,1,4,0,0], 5:[0,3,2,1,1,1,5,0], 6: [0,2,2,1,0,0,2,0], 7: [0,0,0,0,0,0,0,0]}

frame = pd.DataFrame(data)
frame

,0,1,2,3,4,5,6,7
0,0,0,0,0,0,0,0,0
1,0,2,0,2,1,3,2,0
2,0,0,2,0,2,2,2,0
3,0,1,0,1,3,1,1,0
4,0,0,0,1,1,1,0,0
5,0,0,1,3,4,1,0,0
6,0,0,1,0,0,5,2,0
7,0,0,0,0,0,0,0,0


In [4]:
data2 = {0:[4,0,1,1,1,0], 1: [0,5,1,4,1,1], 2:[4,3,4,5,3,1], 3: [3,5,4,3,10,3], 4: [5,4,3,4,4,9], 5:[2,5,3,1,1,3]}

frame2 = pd.DataFrame(data2)
frame2

,0,1,2,3,4,5
0,4,0,4,3,5,2
1,0,5,3,5,4,5
2,1,1,4,4,3,3
3,1,4,5,3,4,1
4,1,1,3,10,4,1
5,0,1,1,3,9,3


In [5]:
data3 = {0:[5,4,1], 1: [5,5,10], 2:[5,4,9]}

frame3 = pd.DataFrame(data3)
frame3

,0,1,2
0,5,5,5
1,4,5,4
2,1,10,9


# HW2

기존 코드

In [6]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda' :
    torch.cuda.manual_seed_all(777)

In [8]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [9]:
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform = transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', train=False, transform = transforms.ToTensor(),
                        download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True, drop_last=True)

In [10]:
class CNN(torch.nn.Module):
    
    def __init__(self):
        
        super(CNN,self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2,stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(4*4*128,625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(self.fc1, torch.nn.ReLU(), torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
   
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [11]:
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [12]:
total_batch = len(data_loader)
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

print('Learning Finished')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.190713644
[Epoch:    2] cost = 0.0520608276
[Epoch:    3] cost = 0.0376487821
[Epoch:    4] cost = 0.0285026077
[Epoch:    5] cost = 0.0243023224
[Epoch:    6] cost = 0.0195404254
[Epoch:    7] cost = 0.0160752013
[Epoch:    8] cost = 0.0142789194
[Epoch:    9] cost = 0.0117072025
[Epoch:   10] cost = 0.0123375095
[Epoch:   11] cost = 0.0106206443
[Epoch:   12] cost = 0.0107046869
[Epoch:   13] cost = 0.00735590933
[Epoch:   14] cost = 0.00858276151
[Epoch:   15] cost = 0.00802615285
Learning Finished


In [13]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

C:\Users\minzz\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\minzz\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9829999804496765


# 2-1

CNN만 수정 해주면 된다. 나머지는 동일한 과정.

In [14]:
class CNN2(torch.nn.Module):
    
    def __init__(self):
        
        super(CNN2,self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32,kernel_size=4, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64,128, kernel_size=3, stride=2, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2,stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(2*2*128,625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(self.fc1, torch.nn.ReLU(), torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
   
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [15]:
model = CNN2().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [16]:
total_batch = len(data_loader)
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

print('Learning Finished')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.258063197
[Epoch:    2] cost = 0.0605403334
[Epoch:    3] cost = 0.0431946777
[Epoch:    4] cost = 0.0337178223
[Epoch:    5] cost = 0.0272049326
[Epoch:    6] cost = 0.0226518922
[Epoch:    7] cost = 0.0182015542
[Epoch:    8] cost = 0.01810693
[Epoch:    9] cost = 0.0134403892
[Epoch:   10] cost = 0.0139215188
[Epoch:   11] cost = 0.0111788521
[Epoch:   12] cost = 0.0117147416
[Epoch:   13] cost = 0.0107532768
[Epoch:   14] cost = 0.00898832921
[Epoch:   15] cost = 0.00780735211
Learning Finished


In [17]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

Accuracy:  0.9886999726295471


# 2-2

(2, 2, 128)

# 2-3

filter의 kernel size가 늘어나고, stride가 커지는 것 모두 output size를 줄이므로, 데이터가 많이 합쳐져서 cost가 늘고 accuracy가 줄을 것 같다.

# 2-4

실제로는 정반대로 accuracy 와 cost 모두 줄어들었다....